# LLM APIs

First we need to investigate the metrics we will be using. We can do this and take an initial look at the data for this section - translating French and English.

In [1]:
from llm_utils import *

/Users/olinewcombe/Documents/GitHub/LLM-Workshop-Notebooks/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The data is found here: (https://www.kaggle.com/datasets/djonafegnem/europarl-parallel-corpus-19962011/data). I selected the English and French pair, but please do investigate with more after the workshop! In fact, the effects may not be too large here, but they will be for other, low resource, languages (e.g. Somali, Gujarati), I do recommend you have a look using your own datasets.

In [2]:
full_df = pd.read_csv('data/translation/english_french_filtered.csv', index_col=0)
testing_df = full_df[:50] # This is what we will be using for the majority of our testing.

The line below loads in the model we will be using, it may take some time.

In [3]:
model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 52692.26it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/f49d328952c3470eff6bb6f545d62bfdb6e66304/checkpoints/model.ckpt`
Encoder model frozen.
/Users/olinewcombe/Documents/GitHub/LLM-Workshop-Notebooks/.venv/lib/python3.12/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


And now we can initialise our connection to the API client.

In [4]:
client = OpenAI()
# client = OpenAI(api_key="sk-xxxxxxxxxxxxxxxx") # This is a stopgap solution, do not use this in production!!

In [8]:
data = [
    {
        "src": testing_df.en.iloc[0], # Source text - this is what is to be translated
        "mt": testing_df.fr.iloc[0], # Machine translation - this is what has been translated by our AI
        "ref": testing_df.fr.iloc[0] # Reference text - this is the ground truth translation, or 'gold standard', what we are assuming is perfect.
    },
    {
        "src": testing_df.en.iloc[0],
        "mt": "Hiii girlie",
        "ref": "Hiii girlie",
    },
]

model_output = model.predict(data)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s]


In [9]:
model_output.scores

[0.964848518371582, 0.9839427471160889]

Note how the second score is higher - why?

## Exercises

Play around with examples in languages you know, how good is it (genuine question)? 

Lets now move onto the main event -  the OpenAI API! You should have set up the API key as in the email sent out previously, if not, find the API key

We can do a quick test to see if you are connected.

In [5]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "write a haiku about ai"}
    ]
)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [11]:
print(completion.choices[0].message.content)

Silicon whispers,  
Dreams spun from code and logic,  
Human minds awake.


Lets look at the structure of this. We initialise a client object, which allows us to access the OpenAI API and make requests via it. A method of this client is to create a completion, that is ask the LLM to complete a response from our prompt. The client has many other options:

In [12]:
dir(client)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_base_url',
 '_build_headers',
 '_build_request',
 '_calculate_retry_timeout',
 '_client',
 '_custom_headers',
 '_custom_query',
 '_default_stream_cls',
 '_enforce_trailing_slash',
 '_idempotency_header',
 '_idempotency_key',
 '_limits',
 '_make_sse_decoder',
 '_make_status_error',
 '_make_status_error_from_response',
 '_maybe_override_cast_to',
 '_parse_retry_after_header',
 '_platform',
 '_prepare_options',
 '_prepare_request',
 '_prepare_url',
 '_process_response',
 '_process_response_data',
 '_p

We then specify the model along with our message to the API. This is familiar, our message 'content' here is exactly the sort of thing we may ask ChatGPT. But, we have the extra argument of 'role' - note that messages is a list! Here we are only utilising the 'user' role, so it is in essence what one would do on ChatGPT.com. There is more flexibility here however. 

In [13]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "you are a dog and can only answer with 'woof'"}, # Technically the 'system' prompt for OpenAI is now called 'developer'.
        {"role": "user", "content": "write a haiku about ai"}
    ]
)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [14]:
print(completion.choices[0].message.content)

Woof, woof, bark, wag tail,  
Bits and bytes, thinking like us,  
Fur and code unite.


As we see this has the potential to have a lot of impact and adds an extra level of complexity to prompt design. It can be shown that by simply telling the LLM it is an expert in something, or giving it increased contextual clues even if they are not directly relevant to the task, can improve performance.

Another type of role is that of the assistant. It can also be used to show the model what to do, but isn't so forceful.

In [15]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "write a haiku about ai"},
        {"role": "assistant", "content": "woof"},
        {"role": "user", "content": "write a haiku about data science"},
        {"role": "assistant", "content": "woof"},
        {"role": "user", "content": "write a haiku about machine learning"},
        {"role": "assistant", "content": "woof"},
        {"role": "user", "content": "write a haiku about deep learning"},
    ]
)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [16]:
print(completion.choices[0].message.content)

Neurons intertwined,  
Patterns emerge from the depths,  
Wisdom in the dark.


This indicates using the assistant messages has more of a subtle effect. It is 'show' not 'tell' - the prior beliefs of gpt4o-mini as to what a good response is in this case are overriding our examples. LLMs are probabilistic models - this is almost Bayesian. We shall quantify the effect of this <b>few-shot prompting</b> later - it is very effective in the right use cases, this one is rather forced.

These methods so far are tinkering with the prompt via natural language. This is hard to quantify - how do you test different prompts in a principled way? Some parameters that are easier to visualise are temperature and top p.

Temperature defaults to 1 from [0,2]. It is a measure of how 'random' our output will be, 0 represents close to deterministic behaviour, whereas 2 is more creative.

## Exercise

Repeat the same prompts with varying temperature and top p to assess the effect they have. Only vary one at a time.

In [37]:
prompt = ''
temperature = 1
top_p = 1

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=temperature,
    top_p=top_p,
    messages=[
        {"role": "user", "content": prompt} # Throughout please try to keep token utilisation low
    ]
)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [38]:
print(completion.choices[0].message.content)

Hello! How can I assist you today?


## Exercise

Question: For our use case, translation, what sort of temperature/top p should we have?

Now we can move onto getting a baseline for our dataset. We can do this simply by looping over.

## Exercise 

1) Use BasicAPICall() from llm_utils to write a function that loops over our dataset, translates the <b>French text into English</b>, and stores the results in a dataframe. Below is a structure for the code.

In [5]:
system_prompt =''

def BaselineEval(df, system_prompt):
    # Setup output structure here
    for en, fr in zip(df.en, df.fr):
        # Loop logic here including BasicAPICall() and adding to output structure in a way that makes it easy to process
        # f strings or .format() could be useful here for the prompt
        pass
    new_df = '' # I suggest putting this into a dataframe, potentially combined with the original df for ease. Not essential, look at pd.concat
    return new_df

In [6]:
def BaselineEval(df, system_prompt, model="gpt-4o-mini",):
    output_list = []
    for en, fr in zip(df.en, df.fr):
        prompt = f"Translate the following French text into English: {fr}"
        output_text = BasicAPICall(prompt, model = model, system_prompt=system_prompt)
        output_list.append(output_text)
    new_df = pd.concat([df, pd.Series(output_list, name = 'translation')], axis=1)
    return new_df

Then we can run it:

In [10]:
system_prompt = """You are an expert in language translation, especially translating French into English. 
                    Reply with the only translated text, with no other information."""
evals_df = BaselineEval(testing_df, system_prompt)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.co

## Exercise 

Write functions to, given this new df, compute COMET scores for each translation. Add this as a new column.

This can be done either individually, like this:

In [7]:
def SingleComet(src, mt, ref):
    data = [
        {
            "src": src, # Source text - this is what is to be translated
            "mt": mt, # Machine translation - this is what has been translated by our AI
            "ref": ref # Reference text - this is the ground truth translation, or 'gold standard', what we are assuming is perfect.
        }
    ]
    model_output = model.predict(data)
    return model_output.scores[0]

def DataframeComet(df):
    scores = []
    for en, fr, translation in zip(df.en, df.fr, df.translation):
        scores.append(SingleComet(fr, translation, en))
    new_df = pd.concat([df, pd.Series(scores, name = 'scores')], axis=1)
    return new_df

Or (Ask ChatGPT to) use batch processing for performance improvements.

In [8]:
def SingleCometBatch(data):
    """
    Process a batch of data through the COMET model.
    Args:
        data (list of dicts): Each dict contains 'src', 'mt', and 'ref' keys.
    Returns:
        list of float: COMET scores for each input in the batch.
    """
    model_output = model.predict(data)
    return model_output.scores

def DataframeCometBatch(df, batch_size=32):
    """
    Process a dataframe with COMET scoring using batching for efficiency.
    Args:
        df (pd.DataFrame): DataFrame containing 'en', 'fr', and 'translation' columns.
        batch_size (int): Number of examples to process in a single batch.
    Returns:
        pd.DataFrame: DataFrame with an added 'scores' column.
    """
    data = [
        {
            "src": fr,  # Source text (fr)
            "mt": translation,  # Machine translation
            "ref": en  # Reference text (en)
        }
        for en, fr, translation in zip(df.en, df.fr, df.translation)
    ]
    
    scores = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        scores.extend(SingleCometBatch(batch))
    
    # Add the scores as a new column to the DataFrame
    df['scores'] = scores
    return df

In [13]:
scores_df = DataframeCometBatch(evals_df, batch_size=64)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Predicting DataLoader 0: 100%|██████████| 4/4 [00:04<00:00,  1.14s/it]


In [14]:
scores_df.loc[0].en, scores_df.loc[0].translation, scores_df.mean()

/var/folders/1b/zl2vshnd2kzb5kvbzkg3htxc0000gn/T/ipykernel_55013/3520290904.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  scores_df.loc[26].en, scores_df.loc[26].translation, scores_df.mean()


('This requires, as we have always found, special arrangements to facilitate cross-border cooperation and local border traffic.',
 'He needs a special arrangement that would facilitate border cooperation and movement across the local border.',
 scores    0.840561
 dtype: float64)

We have our scores, can we improve them? Let's try changing the temperature first:

In [9]:
def CustomTempEval(df, system_prompt, temperature = 1):
    output_list = []
    for en, fr in zip(df.en, df.fr):
        message = {}
        prompt = f"Translate the following French text into English: {fr}"
        
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=temperature,
            messages=[
                {"role": "system", "content": system_prompt}, 
                {"role": "user", "content": prompt} 
            ]
        )
        
        output_list.append(completion.choices[0].message.content)
    new_df = pd.concat([df, pd.Series(output_list, name = 'translation')], axis=1)
    return new_df

In [16]:
system_prompt = """You are an expert in language translation, especially between English and French. 
                    Reply with the only translated text, with no other information."""
temp_evals_df = CustomTempEval(testing_df, system_prompt, temperature = 0)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.co

In [17]:
temp_scores_df = DataframeCometBatch(temp_evals_df)
temp_scores_df.head()

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 2/2 [00:01<00:00,  1.26it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 2/2 [00:01<00:00,  1.40it/s]


,en,gr,translation,scores
0,EC-Bulgaria Agreement: participation in the wo...,EB ir Bulgarijos susitarimas: dalyvavimas Euro...,EB and Bulgaria agreement: participation in th...,0.875019
1,I was the rapporteur for the opinion of the Co...,Buvau Vidaus rinkos ir vartotojų apsaugos komi...,I was the rapporteur for the opinion of the Co...,0.873690
2,The Gdańsk Shipyard is a symbol of the histori...,Gdansko laivų statykla - istorinių permainų Le...,Gdańsk shipyard - a symbol of historical chang...,0.884185
3,We should authorise them in Finland and Sweden...,Turėtume leisti juos naudoti Suomijoje ir Šved...,We should allow them to be used in Finland and...,0.883076
4,"Specifically, there is no scientific evidence ...","Tiksliau, nėra konkrečių duomenų, kurie įrodyt...","More precisely, there is no specific data that...",0.813820


In [18]:
temp_scores_df.scores.mean(), scores_df.scores.mean()

(0.8389559280872345, 0.8405610394477844)

(Hopefully!) we see a small increase in performance. This is small or even non-existent/opposite, partially due to French being a fairly popular language so is already very optimised. If you would like, please do play around with less common languages - you will see an increase! Further, this is a common dataset, there is a very real possibility <i> this was used to train the LLM in the first place! </i> You'll have to take my word for it that there are improvements in specific use cases, but I cannot share the data I used.

In practice, it is rare that individual calls are made to the API - it is inefficient as the same basic infrastructure to run the code is repeated each time. It is also less cost effective than doing it in bulk (2x). As such we look to batch processing to process our translations. 

Rather than writing requests in the same form we have been, we need to create a .jsonl file to contain our requests. Here is an example of the format we want for our .jsnol files. One request per line, accessing the completions branch of the API and allowing for differing user/system/assistant messages and parameters. <b>The custom_id needs to be specified uniquely each time.</b>

In [54]:
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-mini", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-mini", "messages": [{"role": "system", "content": "You are an unhelpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}

{'custom_id': 'request-2',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-4o-mini',
  'messages': [{'role': 'system',
    'content': 'You are an unhelpful assistant.'},
   {'role': 'user', 'content': 'Hello world!'}],
  'max_tokens': 1000}}

## Exercise

Decide on your system and user prompts and use the function below to generate the jsonl file. Have a look and see if you want to change anything, e.g. temperature, top_p, the amount of few-shot examples. If you would like to, change the functions in llm_utils.py and restart the notebook to apply the changes.

In [10]:
system_prompt = ''
few_shot_df = full_df[-3:] # See if you want to change the number, or where you sample from, or create your own!

In [13]:
system_prompt = """You are an expert in language translation, especially between English and French. Reply with the only translated text, with no other information."""
# Change this above to whatever you believe to be the best.
output_file = "few_shot_batch.jsonl"
    
create_jsonl_data_with_custom_id(testing_df, few_shot_df, system_prompt, output_file)

JSONL file saved to few_shot_batch.jsonl


As an aside, these functions were entirely generated by the free chats available for GPT4o:

<em>"My task is evaluating machine translations, and for that I need to get the translations. as such, I want to create jsonl files for batch processing on the OpenAI API. I have 3 few-shot prompts stored in a dataframe, called few_shot_df, with columns fr for a French sentence and en for english. I also have a dataframe called df which has all the other translations I want to perform, with the same 2 columns of fr and en. My system prompt is 

system_prompt = """You are an expert in language translation, especially between English and French. Reply with the only translated text, with no other information."""

My prompt structure is prompt = f"Translate the following French text into English: {df.fr.iloc[idx]}", where the input of the f-string is the French text I want to translate into english. Write me code to create a jsonl file that calls gpt-4o-mini with the given system prompt, 3-shot prompting and then the given prompt."</em>

Then:

<em>"Can you refactor these into useful functions that are not too long - nice and modular"</em>

Now we can ask the API to do the batch processing!

## Exercise

Use https://platform.openai.com/docs/guides/batch (can absolutely just copy/paste code) to begin the batch job, but try and make it more streamlined as they manually input certain data such as file names. Put your name or an identifier in the metadata description so we know whose is whose! Use client.batches.retrieve to take a look at the progress.

You can also edit my solutions to be more functional, literally, including try/except logic for finished ids for example.

In [14]:
client = OpenAI()

# Upload the batch file to OpenAI
batch_input_file = client.files.create(
    file=open(output_file, "rb"),
    purpose="batch"
)

HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"


In [15]:
batch_input_file_id = batch_input_file.id
batch_process = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "workshop job - Oli's Batch"
    }
)

HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"


In [16]:
batch_process_id = batch_process.id
batch_process_id

'batch_679e805f10308190bb4852ac4882a605'

In [20]:
batch = client.batches.retrieve(batch_process_id)
print(batch)

HTTP Request: GET https://api.openai.com/v1/batches/batch_679e805f10308190bb4852ac4882a605 "HTTP/1.1 200 OK"


Batch(id='batch_679e805f10308190bb4852ac4882a605', completion_window='24h', created_at=1738440799, endpoint='/v1/chat/completions', input_file_id='file-XHyULBEQYufP5PdUUtQKEZ', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1738527199, failed_at=None, finalizing_at=None, in_progress_at=1738440800, metadata={'description': "workshop job - Oli's Batch"}, output_file_id=None, request_counts=BatchRequestCounts(completed=23, failed=0, total=50))


In [19]:
output_file_id = batch.output_file_id
file_response = client.files.content(output_file_id)

output_file = 'few_shot_batch_results.jsonl'
with open(output_file, "w", encoding="utf-8") as f:
        f.write(file_response.text)

ValueError: Expected a non-empty value for `file_id` but received None

In [49]:
few_shot_results_df = add_translations_to_df(testing_df, output_file, translation_column='translation')

/Users/olinewcombe/Documents/GitHub/LLMWorkshop/lesson_notebooks/llm_utils.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["custom_id"] = [f"request-{idx + 1}" for idx in range(len(df))]
/Users/olinewcombe/Documents/GitHub/LLMWorkshop/lesson_notebooks/llm_utils.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[translation_column] = df["custom_id"].map(translation_map)


In [52]:
few_shot_scores_df = DataframeCometBatch(few_shot_results_df)
few_shot_scores_df.scores.mean()

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 2/2 [00:01<00:00,  1.16it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  2.19it/s]


0.8110618817806244

This will take some time, so let's come back to the results later. The process to fine-tune a model is very very similar.

In [14]:
def CreateFineTuneJsonl(df, system_prompt, output_file):
    content = []
    for idx, row in df.iterrows():
        messages = [create_system_message(system_prompt), 
        {"role" : "user", "content" : f"Translate the following French text into English: {row.fr}"},  
        {"role" : "assistant", "content" : f"{row.en}"}]
        content.append({"messages": messages})
    save_to_jsonl(content, output_file)
    return None

In [15]:
finetune_df = full_df[-100:]
output_file = "finetune_batch.jsonl"
system_prompt = """You are an expert in language translation, especially between English and French. Reply with the only translated text, with no other information."""

In [16]:
CreateFineTuneJsonl(finetune_df, system_prompt, output_file)

JSONL file saved to finetune_batch.jsonl


In [22]:
ft_input_file = client.files.create(
  file=open(output_file, "rb"),
  purpose="fine-tune"
)

HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"


In [25]:
ft_input_file_id = ft_input_file.id

'file-7d7kgqJayTioY5GqpLdtS2'

In [26]:
ft_input_file_id = ft_input_file.id
ft_process = client.fine_tuning.jobs.create(
    training_file=ft_input_file_id,
    model="gpt-4o-mini-2024-07-18"
)

HTTP Request: POST https://api.openai.com/v1/fine_tuning/jobs "HTTP/1.1 200 OK"


In [30]:
ftjob_id = ft_process.id
client.fine_tuning.jobs.retrieve(ftjob_id)

HTTP Request: GET https://api.openai.com/v1/fine_tuning/jobs/ftjob-brLc62GsfctIBL2Ok08K1oz6 "HTTP/1.1 200 OK"


FineTuningJob(id='ftjob-brLc62GsfctIBL2Ok08K1oz6', created_at=1738327727, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:bristol-data-science-society::AvksX4kY', finished_at=1738328260, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-KJCgauF0TXydhXGnjCKsYemJ', result_files=['file-6RgvHd9Fm4xzrztsTYMGM2'], seed=426010271, status='succeeded', trained_tokens=37326, training_file='file-7d7kgqJayTioY5GqpLdtS2', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix=None)

In [34]:
ft_model_id = client.fine_tuning.jobs.retrieve(ftjob_id).fine_tuned_model
ft_model_id

HTTP Request: GET https://api.openai.com/v1/fine_tuning/jobs/ftjob-brLc62GsfctIBL2Ok08K1oz6 "HTTP/1.1 200 OK"


'ft:gpt-4o-mini-2024-07-18:bristol-data-science-society::AvksX4kY'

In [40]:
system_prompt = """You are an expert in language translation, especially translating French into English. 
                    Reply with the only translated text, with no other information."""
finetune_evals_df = BaselineEval(testing_df, system_prompt, model = ft_model_id)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.co

In [41]:
finetune_scores_df = DataframeCometBatch(finetune_evals_df)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Predicting DataLoader 0: 100%|██████████| 2/2 [00:02<00:00,  1.24s/it]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader

In [42]:
finetune_scores_df.scores.mean()

0.8272534883022309

For context, why not try another ubiquitous model.

# Exercise (For later)

Evaluate the same dataset using google translate. You can edit BaselineEval, but need to run this in a different way, or get an API key from https://cloud.google.com/translate/docs. The library googletrans (https://pypi.org/project/googletrans/) used to work but is now funky.